## Loading the model to classify

This notebook provides a quick example on how to use the package and apply it to hydrodynamical simulations. Firstly, we load a minimum set of libraries.

In [8]:
import joblib


#I am loading pandas to create an array, but you can use your preferred library
import pandas as pd
import numpy as np

We load the Random Forest classifier trained to sort star particles between BCG and ICL. 

In [23]:
model=joblib.load('Best_Model-Random-3par.pkl')


/Users/imarini/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.23.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/imarini/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.23.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


## Preparing the input data.

We need to provide to the Random Forest a matrix of size (3, N) where N is the number of star particles to classify. The data needs to be provided in a very specific order, as the model was trained to receive it. Thus, we need to pass:

1. Clustercentric-distance (position) of the star particle normalised for $R_{200}$ (critical) of the halo;
2. Rest-frame velocity of the star particle normalised for $V_{200}$ (critical) of the halo:
3. Logarithmic halo mass $M_{200}$.

A few remarks. The clustercentric distance is intended as the module of the vector centered on the center of mass of the halo ($x_0, y_0, z_0$) and directed to the position of the star particle ($x,y,z$). Thus, in mathematical form,

    r = sqrt((x-x0)^2+(y-y0)^2+(z-z0)^2).
    
The rest-frame velocity is with respect to the velocity of the particles in the halo. On the other hand,
 $V_{200}$ = sqrt(G * $M_{200}$/$R_{200}$)
where G is the gravitational constant, $M_{200}$ is the halo mass within a overdensity 200 times the critical density of the universe and $R_{200}$ is the corresponding radius. 
Finally, the last parameter is given as the logarithm of the halo mass $M_{200}$ normalised for a nominal mass $M_{0}$ which we assumed to be $M_{0} = 10^{14} M_{\odot}$.

For illustrative purposes I am going to create a sample of data. Notice that this does not replicate a physically possible system.

In [19]:
#Gravitational constant
G = 4.30091e-6 #kpc Msun^-1 (km/s)^2


#Let us classify N stars
N = 20
r = np.random.randn(N)*100 +300    #kpc
v = np.random.randn(N)*5+100       #km/s

M200 = 2.98e13                     #Msun
R200 = 464.97                      #kpc 
V200 = np.sqrt(G*M200/R200)        #km/s

Create mock array to classify

In [20]:
Data = pd.DataFrame({'distance':np.log10(r/R200),'velocity':v/V200,'logm200':[np.log10(M200*1e-14)]*len(v)})

#Need to remove all infinite values
Data = Data.replace([np.inf, -np.inf], np.nan).dropna(axis=0)


## Results of the classification

The classifier returns a binary array of the same size of the input file. The binary classification defines the outcome of the sorting for each particle.

See below:
- predictions == 1 --> the particle is bound to the BCG
- predictions == 0 --> the particle is bound to the ICL

In [24]:
final_predictions = model.predict(Data)



/Users/imarini/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


Use the results as a flag to mask or not mask your particles

In [26]:
print (final_predictions)

[0. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
